Twitter Streaming

Importiamo tutte le dipendenze necessarie

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

Impostiamo le credenziali di Twitter

In [2]:
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_token = 'XXX'
access_secret = 'XXX'

Criamo un TweetsListener, che sarà responsabile dello streaming.
Per l'inizializzazione, forniamo un oggetto socket e implementiamo due metodi: on_data e on_error.
Il primo metodo è responsabile della ricezione dei dati dal flusso di Twitter e dell'invio a un socket.
Il secondo viene utilizzato per ricevere messaggi di errore.

In [3]:
class TweetsListener(StreamListener):
  def __init__(self, csocket):
    self.client_socket = csocket
  def on_data(self, data):
      try:
        msg = json.loads( data )
        print( msg['text'].encode('utf-8') )
        self.client_socket.send( 
        msg['text'].encode('utf-8') )
        return True
      except BaseException as e:
        print("Error on_data: %s" % str(e))
      return True
  def on_error(self, status):
    print(status)
    return True

Ci colleghiamo allo streaming di Twitter. Per fare ciò, creiamo un metodo sendDate (che avrà come parametro un oggetto socket) dove autorizziamo con le nostre credenziali e creiamo un'istanza dell'oggetto Stream.

In [4]:
def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track=['coronavirus'],languages=["en"])

Criamo un oggetto socket e avviamo il processo di streaming.

In [5]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)   #Creo un oggetto socket    
host = socket.gethostbyname(socket.gethostname())      
port = 15000                 
s.bind((host, port))  #Lego macchina locale e porta  
s.listen(5)           #Attendo una connessione client         
print("Server inizializzato, ora in ascolto...")

Server inizializzato, ora in ascolto...


In [6]:
client, address = s.accept() #Stabilisco una connessione con il client  
print(f"Richiesta ricevuta da: {address}")

Richiesta ricevuta da: ('192.168.56.1', 64423)


In [7]:
sendData(client) #Chiamo il metodo sendData per creare una nuova connessione

b'RT @sardanarohit: 97,000 US children test positive for coronavirus in two weeks as schools open https://t.co/MhpRmHR0f4'
b'RT @_bussey23: How you can force a vaccine but cant force a mask'
b"RT @repmarkpocan: This week's coordinated campaign brought to you by: science &amp; logic."
b'RT @nytimes: 200,000 more people have died in the U.S. since March than would be expected in a normal year.\n\nThis number, about 60,000 high\xe2\x80\xa6'
b'RT @NBCNews: A Georgia school district that does not require masks has closed a high school and now has over 1,100 students and staff in qu\xe2\x80\xa6'
b'RT @_bussey23: How you can force a vaccine but cant force a mask'
b'RT @IndependentSage: The UK economy is officially in recession. How will we cope with this and a potential second wave of coronavirus?\n\nAre\xe2\x80\xa6'
b'What we know about the risks of coronavirus from food and packages - National | https://t.co/LSl7IcSd0i https://t.co/KTd8fGzFLy'
b'RT @MidwinCharles: Are we great yet?'
b'RT @D

KeyboardInterrupt: 